# Introduction à MT avec OpenNMT.

![OPENNMT](https://avatars.githubusercontent.com/u/23035727?s=200&v=4)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RobotsMali-AI/rmai/blob/master/Introduction_MT_OpenNMT.ipynb)

Présenté par:

- Sebastien Diarra, Michael Leventhal

Procédure :

- IndabaX Mali 2022

## Agenda
- À savoir
- Introduction
- Prérequis
- Configuration de l'Environnement
- Préparation des données
- Architecture du modèle
- Entraînement
- Essai & Évaluation des performances [BLEU, Tensorboard]
- Questions et Réponses


## Buts et objectifs
- Pour que vous compreniez le MT. Pipeline
- Apprenez à configurer l'environnement OpenNMT
- Pour vous intéresser à notre initiative Bayɛlɛmabaga & MT pour les langues locales

**REMARQUES IMPORTANTES** :

- Cette pratique est destinée à ceux qui n'ont aucune expérience en MT ou ML d'ailleurs.
- La pratique est davantage axée sur l'action plutôt que sur le récit.
- J'espère que vous quitterez cette session avec un désir de contribuer au travail de MT pour le Mali.
- Vous n'avez pas besoin d'être un expert pour contribuer. Vous pouvez contribuer Data !

*NB* : Joignez-vous à quelqu'un avec un ordinateur si vous n'avez pas apporté le vôtre.

## Introduction

Dans cette pratique, nous découvrirons OpenNMT et comment former un modèle de traduction automatique bilingue pour le bambara - français. Ceci est un tutoriel d'introduction pour se mouiller les pieds. Une approche pratique est fortement recommandée, tous les participants sont encouragés à suivre. Le code source de cet atelier peut être exécuté via Colab à l'URL suivante :

[Notebook](https://colab.research.google.com/github/RobotsMali-AI/rmai/blob/master/Fr_Introduction_MT_OpenNMT.ipynb)


### [OpenNMT](https://opennmt.net/)

"OpenNMT - est un écosystème open source pour la traduction automatique neuronale et l'apprentissage des séquences neuronales"

C'est un framework, ou toolkit, qui permet d'entrainer un modèle de langage.

OpenNMT est pour :
- Débutants
- Amateur
- Experts

### MT (Traduction Automatique)

Processus automatisé de traduction d'une langue à une autre par machine sans intervention humaine. La traduction est généralement effectuée par un programme informatique. Il existe différents types de MT :

- MT basée sur des règles : règles de grammaire et de langue utilisées comme base pour un logiciel pour effectuer la traduction.

![RULE-IMAGE](https://i.ibb.co/7XbQV0H/grammar-rules.png)

- SMT : MT statistique : apprentissage basé sur la distribution de probabilité.


<div>
<img alt="SMT" width="300"; height="250" src="https://www.researchgate.net/profile/Andy-Way/publication/220418877/figure/fig5/AS:669019598245898@1536518110475/A-Statistical-Machine-Translation-System-adapted-from-Figure-1-in-Brown-et-al-13.png" />

<sub>Courtesy of: https://www.researchgate.net/publication/220418877_Hybrid_data-driven_models_of_machine_translation</sub>
</div>

- NMT : Neural Machine Translation : Apprentissage basé sur les réseaux de neurones artificiels.

**NMT : c'est ce qui nous intéresse dans cet atelier**

### NMT

Est-ce l'utilisation de techniques d'apprentissage en profondeur (réseaux de neurones) pour former un modèle de traduction. Cette approche est devenue l'approche dominante pour faire de la traduction automatique. NMT s'est avéré très efficace dans la traduction automatique ces dernières années.

<div>
<img width="300" height="250" src="https://1.cms.s81c.com/sites/default/files/2021-01-06/ICLH_Diagram_Batch_01_03-DeepNeuralNetwork-WHITEBG.png">
</div>

OpenNMT -- nous aide à expérimenter rapidement la NMT, ainsi qu'à créer des systèmes/solutions de MT en moins de temps.

NMT englobe les deux autres approches dans une certaine mesure.

**NB** : ***Venez à l'événement PNL samedi, pour apprendre ces concepts en détail.***

## Prérequis
- Compréhension de la CLI Unix (aide)
- Compréhension de Python (JupyterNB)
- Les concepts de base de ML (tels que Pipeline, Models, NN) aident mais ne sont pas obligatoires



## Configuration de l'Environnement

### Paquets

- `OpenNMT-py` : paquet opennmt principal (version pytorch)
- `rmaipkg` : Package pour les ensembles de données et les modèles de RobotsMaliAI (Later)
- `Daba` (non requis) : une version modifiée de [daba](https://github.com/maslinych/daba)

In [ ]:
%pip install OpenNMT-py subword_nmt sentencepiece # Main OpenNMT Package (No Need for subword_nmt sentencepiece)
%pip install --no-cache-dir https://github.com/RobotsMali-AI/rmai/releases/download/0.0.3/rmaipkg-0.0.3.tar.gz # RobotsMaliAI's Datasets and Models
%pip install -U https://github.com/s7d11/daba/releases/download/v0.0.1-alpha/daba-0.9.2.tar.gz # Non-UI Version of Daba
%pip install sacrebleu # Redundant comes with OpenNMT-py

### Configuration de l'espace de travail (Google Colab)

- Qu'est-ce que Colab ?

- Local vs Google Drive
    - Experimentation vs Persistence

In [ ]:
import os
import pathlib
# TODO: validate that user works on Colab

# Select your environment
WORK_ENV = 0 # 0=Local 1=Drive

CWD = os.getcwd()
WORK_DIR_Lo = CWD+"/onmt"#"/content/onmt"

if(WORK_ENV):
    if os.path.exists("/content"):
        from google.colab import drive
        WORK_DIR_Dr = "/content/drive/MyDrive/onmt"
        drive.mount('/content/drive') # Mounting your Google drive
    else:
        print("Colab not detected. Reverting to local environment")
        WORK_DIR_Dr = WORK_DIR_Lo

WORK_DIR = WORK_DIR_Dr if WORK_ENV else WORK_DIR_Lo
DATA_DIR = f"{WORK_DIR}/data"

if(not os.path.exists(WORK_DIR)):
    !mkdir -p $DATA_DIR
    print(f"Environment Created: {WORK_DIR}")
else:
    print("Environment already exists...")

os.chdir(WORK_DIR) # Naviting to Work-environment

### Activer le GPU

- Édition > Paramètres du bloc-notes > GPU > Enregistrer
- Exécution > GPU


**IMPORTANT** : Gardez le GPU détaché jusqu'à ce que vous fassiez quelque chose qui nécessite des GPU (c'est-à-dire de l'entrainement)

In [ ]:
# GPU's information

!nvidia-smi 

## Préparation des données

`rmaipkg` a des textes Bambara parallèles que nous utiliserons pour cette étape. Vous échangez facilement cela avec votre source de données.


Sachez à l'avance : ***Le texte n'est pas entièrement nettoyé si vous rencontrez des problèmes, il est préférable de générer un échantillon différent***

In [ ]:
# Import the 'parralel' module from rmaipkg
from rmai.datasets.text import parallel

texts = parallel.get_text(max_len=10000, randomize=True) # Load texts in memory

# Show first 10 lines
print(texts[:10])

- `parallel` : module pour les textes parallèles de RobotsMali
- `parallèle.get_text` :
    - retourne `max_len` nombre de lignes
    - `randomize` lorsqu'il est défini, mélange les textes. Bon pour générer un ordre différent pour les textes à chaque exécution.

In [ ]:

train, valid = parallel.random_split(texts, 80) # Split Data into training and validation 80/20

print(
    "Splitted correctly: ", 
    len(train)+len(valid) == len(texts)) # Sanity Check
 
"""
TASK I: Create the following variables containing unique language for each set
- trainbam: train set bambara
- trainfra: train set francais
- validbam: valid set bambara
- validfra: valid set francais
"""

# CODE HERE (Discuss in pseudo code)

- `parallel.random_split` : divise l'objet texte en ratios pour l'entraînement et la validation, il renvoie un type de tuple.
    - Le premier argument est les "textes"
    - Le deuxième argument est le taux auquel c'est divisé

In [ ]:
#@title
# Task I - Solution
x_extractor = lambda x, dset: [i[x] for i in dset]

trainbam = x_extractor(0, train)
trainfra = x_extractor(1, train)
validbam = x_extractor(0, valid)
validfra = x_extractor(1, valid)

- `x_extractor` est une fonction lambda pour la tâche de séparer les valeurs.

In [ ]:
# Optional Work - Further Cleaning the Data

import string

def clean_lines(lines):
  nlines = []
  for i in lines:
    for j in string.punctuation + "«»":
      if(not j in r"!.:?()[]"):
        i = i.replace(j, " ")
    nlines.append(i)
  return " ".join("".join(nlines).strip().split())

trainbam = [clean_lines(i) for i in trainbam]
trainfra = [clean_lines(i) for i in trainfra]
validbam = [clean_lines(i) for i in validbam]
validfra = [clean_lines(i) for i in validfra]


- Écrivons maintenant nos données dans des fichiers réels sur le système. `rmaipkg` a une méthode appelée `write_to` qui le fait pour nous.

In [ ]:
parallel.write_to(lines=trainbam, name="src-train", path=DATA_DIR)
parallel.write_to(lines=trainfra, name="tgt-train", path=DATA_DIR)
parallel.write_to(lines=validbam, name="src-val", path=DATA_DIR)
parallel.write_to(lines=validfra, name="tgt-val", path=DATA_DIR)


Maintenant que nous avons écrit nos données et que nous avons un ensemble de données, adaptons-le maintenant à la traduction automatique.

In [ ]:

model_name = "bam2fr" # Name you are giving to your model
config_name = f"{WORK_DIR}/cool_config.yaml"

config = f"""

# IndabaX - Mali 2022 (Configuration)

save_data: {model_name}/run/{model_name}

overwrite: True # Toggle this for rewritting

data:  
    robotsmali:
        path_src: data/src-train.txt # DataPath
        path_tgt: data/tgt-train.txt
        transforms: []
        weight: 1
    valid:
        path_src: data/src-val.txt
        path_tgt: data/tgt-val.txt
        transforms: []
        weight: 1

src_seq_length: 250
tgt_seq_length: 250

src_vocab: {model_name}/run/{model_name}.vocab.src.txt
tgt_vocab: {model_name}/run/{model_name}.vocab.tgt.txt
skip_empty_level: silent

"""

# Write config file
with open(config_name, "w") as fp:
    fp.write(config)


Ici, nous avons créé un fichier de configuration `cool_config.yaml` contenant des informations générales sur le jeu de données. Qu'avons-nous fait:

- Nous avons attribué le nom de notre modèle à une variable
- Générez un fichier de configuration avec quelques détails de base tels que :
    - où se trouvent les données
    - où générer les fichiers de vocabulaire

In [ ]:
# OpenNMT Building the vocabulary

!onmt_build_vocab -c $config_name -n_sample -1 --dump_samples # -1 full corpus, bpe, sentencepiece

La commande `onmt_build_vocab` construit le vocabulaire nécessaire à la formation. Ici, nous l'avons appelé avec trois paramètres

- `-c config_file` : chemin du fichier de configuration à utiliser pour le processus de création de vocabulaire

- `-n_sample -1` : la construction d'un vocabulaire à l'aide du corpus complet peut remplacer **-1** par n'importe quel nombre, lorsque l'argument est omis, sa valeur par défaut est 5000.

- `-- dump_sampes` : (argument facultatif) Écrire des échantillons lors de la construction du vocabulaire, trouvés dans `model_name/run/model_name`

Nous n'avons pas besoin de construire un vocabulaire en utilisant le `onmt_build_vocab`, nous pouvons le construire nous-mêmes souvent cela est nécessaire, `tokenization` varie selon la langue, chaque langue a ses caractéristiques uniques. Un exemple de ceci est montré ci-dessous.

In [ ]:
# Custom Tokenization (rm-daba-tokenizer) 
# - Shown as example, CELL can be skipped

from rmai.utils import daba

dabautil = daba.DabaUtils()
dabautil.tokenize_line(trainbam+validbam)[:20]

[`Daba`](https://github.com/maslinych/daba) est un outil incroyable développé par des linguistes de l'INALCO. Daba est une ceinture à outils pour les linguistes, et par conséquent pour nous, les gens des sciences informatiques qui cherchons à travailler avec la langue bambara.

- Si vous cherchez à travailler avec Bambara, nous vous le recommandons fortement.


(Retour à OpenNMT)

### Entraînement


#### Architecture d'un modèle

Nous "concevons" le modèle à travers la configuration que nous avons générée précédemment. Dans la cellule de code ci-dessous, nous écrivons comment nous nous attendons à ce que notre modèle soit.

*Bien que le fichier de configuration puisse être mis à jour et ajusté.* Nous limiterons notre modification à quelques options, car nous n'avons pas assez de temps pour en discuter.

Vous pouvez appeler cette configuration comme ingrédients de la sorcière.

- save_model : est le chemin dans lequel le point de contrôle de notre modèle est enregistré.
- save_checkpoint_steps : X, à chaque étape X de l'entraînement, un point de contrôle est enregistré.
- train_steps : Nombre d'étapes d'entraînement
- valid_steps : exécuter la validation, toutes les X étapes
- warmup_step : étape permettant d'ajuster le taux d'apprentissage après le début de l'entraînement.
- GPU : basculer si vous utilisez GPU ou CPU

BTW : Il s'agit d'une architecture de modèle de transformateur (transformer).

Inspiré de :
- [https://github.com/OpenNMT/OpenNMT-py/blob/master/config/config-transformer-base-1GPU.yml](https://github.com/OpenNMT/OpenNMT-py/blob/master/config/config-transformer-base-1GPU.yml)
- Travail effectué par [A. A. Tapo](https://github.com/israaar)

In [ ]:
save_freq =  5 #100 # Checkpoint saving steps
training_steps = 10 #500 # Number of steps for the training
valid_steps = 2# 50 # Steps to validate training perfomance
warmup_step = 3 #125 # Warmup 1/4th of training total
GPU = 0 # 0: CPU or 1:GPU

config += f"""

# Training
save_model: {model_name}/{model_name}
save_checkpoint_steps: {save_freq}
keep_checkpoint: 3 # Better for this activity
seed: 1234
train_steps: {training_steps}
valid_steps: {valid_steps}
warmup_steps: {warmup_step}
report_every: {int(training_steps / 5)}
external_evaluators: BLEU
tensorboard: true
tensorboard_log_dir: {model_name}/logs

# Model
encoder_type: transformer
decoder_type: transformer
word_vec_size: 256 # Word embedding size
hidden_size: 256
layers: 6
transformer_ff: 1024
heads: 4
dropout: 0.3

# HyperParams
accum_count: 8
optim: adam
adam_beta1: 0.9
adam_beta2: 0.998
decay_method: noam
learning_rate: 0.0004
max_grad_norm: 0.0

batch_size: 4096
batch_type: tokens
normalization: tokens
label_smoothing: 0.2

max_generator_batches: 0

param_init: 0.0
param_init_glorot: 'true'
position_encoding: 'true'

"""

if(GPU):
    config += """
world_size: 1 # CPU
gpu_ranks: [0] # Change to number of GPU if more than 1 GPU
"""

"""
TASK II - Rewrite the config with this new configuration
"""

# CODE HERE

In [ ]:
#@title

with open(config_name, "w") as fp:
    fp.write(config)


- **NOW LET THE MAGIC BEGIN**

Nous allons exécuter notre expérience, mais avant de lancer la commande d'entraînement, nous allons démarrer tensorboard. ***Tensorboard*** nous aide à visualiser ce qui se passe avec notre modèle de formation.

**Important** : Si vous obtenez une erreur `403`. Activez Cookie / Autorisez le navigateur à afficher le Tensorboard.

In [ ]:
%load_ext tensorboard

%tensorboard --logdir bam2fr/logs/

Le `onmt_train` prend le fichier de configuration comme argument. Espérons que si nous avons tout fait avant, la formation devrait commencer sans aucun problème. La durée de la formation dépend des étapes de formation, de la puissance de calcul et de data_size.

In [ ]:
!onmt_train -config $config_name

## Essai & Évaluation des performances [BLEU, Tensorboard]

### Task 3
- Créer son propre fichier de test (10 phrases)
    - Créez un nouveau fichier with dans le répertoire `data` puis écrivez 10 lignes bambara sur le fichier.
    - Créez un autre fichier dans le répertoire `data` puis écrivez la traduction française du fichier texte en bambara.

### Vous pouvez utiliser notre kit de test si vous vous sentez paresseux ou sans inspiration

- Copiez notre fichier de test à partir de [Repo](https://github.com/robotsmali-ai/rmai)

In [ ]:
#@title
# Test Set
bamf = "https://raw.githubusercontent.com/RobotsMali-AI/rmai/master/mbam.txt"
fraf = "https://raw.githubusercontent.com/RobotsMali-AI/rmai/master/mfr.txt"

!wget {bamf} {fraf} -P data # Bam-Fra


La commande `onmt_translate` est utilisée pour effectuer la traduction avec le modèle formé. Ici, nous utilisons cinq paramètres :
- `model` : un fichier de point de contrôle de modèle
- `src` : notre fichier de test la langue source
- `output` : où nous voulons écrire les prédictions de notre modèle
- `gpu` : nombre de GPU sur lesquels exécuter
- `verbose` : affiche les scores par prédiction

In [ ]:
ckpt_step = 10
tsrc = "mbam.txt" # Change this to your bambara test file

selected_model_name = f"{model_name}_step_{ckpt_step}.pt"
output = f"{model_name}/pred_{model_name}_{ckpt_step}.txt"

!onmt_translate -model $model_name/$selected_model_name -src data/$tsrc -output $output # -gpu -1 -verbose

**Ta-da! Let's see how our model did, now.**

In [ ]:
!cat $output

Le score de Bleu `(BiLingual Evaluation Understudy)` est une mesure automatique pour évaluer la traduction automatique. C'est utile, car cela nous permet d'avoir une idée générale de la façon dont notre modèle fonctionne si nous travaillons avec un modèle plus grand et une taille de données plus grande.

Dans notre cas, nous utiliserons notre propre traduction comme référence pour déterminer les performances de notre modèle.

In [ ]:
reference = "data/mfr.txt"

!sacrebleu $reference -i $output -m bleu -b -w 4

Prochaine étape : revenez en arrière et ajustez les paramètres pour votre propre personnalisation.

>>> **Merci de nous avoir rejoint. Venez à l'événement N.L.P samedi pour en savoir plus sur NLP, ASR en détails.**

<div>

<img width="250" src="https://sp-ao.shortpixel.ai/client/to_webp,q_glossy,ret_img/https://indabax.robotsmali.org/wp-content/uploads/2022/11/robotsmali.png" />

</div>